In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import demo
from pathlib import Path
from pcdet.utils import common_utils
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network, load_data_to_gpu
import pcdet.datasets.processor.data_processor as DP
import pcdet.datasets.dataset as D


ckpt_path = './../output/kitti_models/voxel_rcnn_car/A_S22/ckpt/checkpoint_epoch_160.pth'
cfg_path = './cfgs/kitti_models/voxel_rcnn_car.yaml'
dataset_cfg = './cfgs/dataset_configs/kitti_dataset_copy.yaml'
data_path = '/home/jiazx_ug/dataset/S2/A/training/velodyne/128976.bin'
#data_path ='/home/jiazx_ug/dataset/S4/temp/training/snow_01.bin'
txt_path = '/home/jiazx_ug/dataset/S2/A/training/label_2/128976.txt'
data_dir_path = '/home/jiazx_ug/dataset/S4/temp/training/'


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# use the parameter in the config file, voxelize the point cloud

# 1. load parameters
logger = common_utils.create_logger() 
cfg_from_yaml_file(cfg_path, cfg)
cfg_from_yaml_file(dataset_cfg, cfg.DATA_CONFIG)

class_names = cfg.CLASS_NAMES
point_cloud_range = cfg.DATA_CONFIG.POINT_CLOUD_RANGE
training = False
num_point_features = cfg.DATA_CONFIG.DATA_AUGMENTOR.AUG_CONFIG_LIST[0].NUM_POINT_FEATURES
dataset_processor_cfg = cfg.DATA_CONFIG.DATA_PROCESSOR
voxel_size = dataset_processor_cfg[2].VOXEL_SIZE
max_number_per_voxel = dataset_processor_cfg[2].MAX_POINTS_PER_VOXEL
points_range = cfg.DATA_CONFIG.POINT_CLOUD_RANGE


# demo_dataset = demo.DemoDataset(
#     dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
#     root_path=Path(data_path), ext='.bin', logger=logger
# )

# model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=demo_dataset)
# model.load_params_from_file(filename=ckpt_path, logger=logger, to_cpu=True)


In [3]:
# (backbone_3d): VoxelBackBone8x(
#       (conv_input): SparseSequential(
#         (0): SubMConv3d(4, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
#         (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
#         (2): ReLU()
#       )

In [4]:
import numpy as np
from skimage import transform
from pcdet.models.backbones_3d.spconv_backbone import VoxelBackBone8x
import torch
import torchvision
import spconv
tv = None
try:
    import cumm.tensorview as tv
except:
    pass
class voxelgenerator():
    def __init__(self,vsize_xyz, coors_range_xyz, num_point_features, max_num_points_per_voxel, max_num_voxels):
        print(vsize_xyz, coors_range_xyz, num_point_features, max_num_points_per_voxel, max_num_voxels)
        try:
            from spconv.utils import VoxelGeneratorV2 as VoxelGenerator
            self.spconv_ver = 1
        except:
            try:
                from spconv.utils import VoxelGenerator
                self.spconv_ver = 1
            except:
                from spconv.utils import Point2VoxelCPU3d as VoxelGenerator
                self.spconv_ver = 2

        if self.spconv_ver == 1:
            self._voxel_generator = VoxelGenerator(
                voxel_size=vsize_xyz,
                point_cloud_range=coors_range_xyz,
                max_num_points=max_num_points_per_voxel,
                max_voxels=max_num_voxels
            )
        else:
            self._voxel_generator = VoxelGenerator(
                vsize_xyz=vsize_xyz,
                coors_range_xyz=coors_range_xyz,
                num_point_features=num_point_features,
                max_num_points_per_voxel=max_num_points_per_voxel,
                max_num_voxels=max_num_voxels
            )

    def generate(self, points):
        if self.spconv_ver == 1:
            voxel_output = self._voxel_generator.generate(points)
            if isinstance(voxel_output, dict):
                voxels, coordinates, num_points = \
                    voxel_output['voxels'], voxel_output['coordinates'], voxel_output['num_points_per_voxel']
            else:
                voxels, coordinates, num_points = voxel_output
        else:
            assert tv is not None, f"Unexpected error, library: 'cumm' wasn't imported properly."
            voxel_output = self._voxel_generator.point_to_voxel(tv.from_numpy(points))
            tv_voxels, tv_coordinates, tv_num_points = voxel_output
            # make copy with numpy(), since numpy_view() will disappear as soon as the generator is deleted
            voxels = tv_voxels.numpy()
            coordinates = tv_coordinates.numpy()
            num_points = tv_num_points.numpy()
        return voxels, coordinates, num_points
    
    def meanVFE(self,voxel_features, voxel_num_points):
        points_mean = voxel_features[:, :, :].sum(dim=1, keepdim=False) # Use sum() instead of _sum()
        normalizer = torch.clamp_min(voxel_num_points.view(-1, 1), min=1.0).type_as(voxel_features)
        points_mean = points_mean / normalizer #求出每个体素内 sum / 点数
        return points_mean.contiguous()
    




voxel_size = dataset_processor_cfg[2].VOXEL_SIZE
points_range = cfg.DATA_CONFIG.POINT_CLOUD_RANGE
num_point_features = cfg.DATA_CONFIG.DATA_AUGMENTOR.AUG_CONFIG_LIST[0].NUM_POINT_FEATURES
max_number_per_voxel = dataset_processor_cfg[2].MAX_POINTS_PER_VOXEL
max_number_of_voxels = dataset_processor_cfg[2].MAX_NUMBER_OF_VOXELS['train']
#grid_size = cfg.MODEL.ROI_HEAD.ROI_GRID_POOL.GRID_SIZE

grid_size = points_range[3:6] - points_range[0:3] / np.array(voxel_size)
grid_size = np.round(grid_size).astype(np.int64)
print(grid_size)


voxel_generator = voxelgenerator(voxel_size, points_range, num_point_features, max_number_per_voxel, max_number_of_voxels)
points = np.fromfile(data_path, dtype=np.float32).reshape(-1, 4)
# only use the points in points range
# print('origin', len(points))
# points = points[(points[:, 0] >= points_range[0]) & (points[:, 0] < points_range[3]) &
#                 (points[:, 1] >= points_range[1]) & (points[:, 1] < points_range[4]) &
#                 (points[:, 2] >= points_range[2]) & (points[:, 2] < points_range[5])]
# print('after', len(points))


voxels, coordinates, num_points = voxel_generator.generate(points)

voxels = torch.from_numpy(voxels)
coordinates = torch.from_numpy(coordinates)
num_points = torch.from_numpy(num_points)

# print max of coordinates
print(coordinates.max(0), coordinates.min(0))

mean_voxel = voxel_generator.meanVFE(voxels, num_points)
print(mean_voxel.shape, coordinates.shape, num_points.shape)


# 2. load the voxelbackbone
backbone = VoxelBackBone8x(model_cfg=cfg, input_channels=num_point_features, grid_size=grid_size)
# [batch_idx, z_idx, y_idx, x_idx]
coordinates = torch.cat([torch.zeros_like(coordinates[:, :1]), coordinates], dim=1)
for i in range(len(coordinates)):
    coordinates[i, 0] = i

# mean_voxels, coordinates, num_points to cuda
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)
mean_voxel = mean_voxel.to(0)
coordinates = coordinates.to(0)
num_points = num_points.to(0)

sp_input_res = backbone.get_input_layer_result(voxel_features=mean_voxel, voxel_coords=coordinates,batch_size=1)


[1478  840   31]
[0.05, 0.05, 0.1] [-70.4, -40, -3, 70.4, 40, 1] 4 5 16000


FileNotFoundError: [Errno 2] No such file or directory: '/home/jiazx_ug/dataset/S2/A/training/velodyne/128976.bin'

In [ ]:
sp_input_res.features

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0687, 0.6572],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0513, 0.6365],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0514, 0.6329],
        ...,
        [0.0000, 0.6461, 0.0000,  ..., 0.0000, 0.3437, 0.0000],
        [0.0000, 0.6556, 0.0000,  ..., 0.0000, 0.3320, 0.0000],
        [0.0000, 0.6567, 0.0000,  ..., 0.0000, 0.3285, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)

In [ ]:
# 根据sp_input_res.indices(batch_id,z,y,x)生成索引三维数组，填充为该位置的batch_id
sp_cpu = sp_input_res.features.cpu().detach().numpy()

indices = sp_input_res.indices.int().cpu().numpy()
indices_3d = np.zeros(grid_size, dtype=np.int32)

ind_temp = indices.copy()
# 不确定这里是否正确
ind_temp[:, 1:] = ind_temp[:, 1:] / 2
ind_temp = ind_temp.astype(np.int32)
indices_3d[ind_temp[:, 3], ind_temp[:, 2], ind_temp[:, 1]] = ind_temp[:, 0]


In [ ]:
print(max(indices[:,1]), min(indices[:,1]))
print(max(indices[:,2]), min(indices[:,2]))
print(max(indices[:,3]), min(indices[:,3]))

39 2
1599 0
2813 0


In [ ]:
# N, C, D, H, W = spatial_features.shape
# spatial_features = spatial_features.view(N, C * D, H, W)


In [ ]:
# # spatial_features = spatial_features.cpu().detach().numpy()
# print(spatial_features.shape)
# print(grid_size)

# for i in range(512):
#     print(spatial_features[0, i, :, :])
